In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import gradio as gr
import openai
import json
import os
import csv
import pandas as pd

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except ValueError:
        return False
    
def create_csv_file(file_path, data):
    # Check if the CSV file already exists
    file_exists = os.path.isfile(file_path)

    # Open the CSV file in append mode
    with open(file_path, 'a', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=data.keys())

        # Write the header row if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write the data row
        writer.writerow(data)
        
def EXTRACT_INFO(data):
    # Initialize variables
    num_entries = len(data)
    total_pos = 0
    total_neg = 0
    total_neu = 0
    make_appointment_count = 0
    appointment_scheduled_count = 0
    make_appointment_failure_count = 0
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    schedule_failure_reasons = {}

    # Analyze the data
    for entry in data:
        sentiment = entry['sentiment_of_call']
        reason = entry['reason_for_calling']
        appointment_scheduled = entry['appointment_scheduled']
        reason_for_schedule_failure = entry['reason_for_schedule_failure']

        if sentiment == 'positive':
            total_pos += 1
        elif sentiment == 'negative':
            total_neg += 1
        elif sentiment == 'neutral':
            total_neu += 1

        if reason == 'make_appointment':
            make_appointment_count += 1
            if sentiment == 'positive':
                positive_count += 1
            elif sentiment == 'neutral':
                neutral_count += 1
            elif sentiment == 'negative':
                negative_count += 1

            if appointment_scheduled == "FALSE":
                make_appointment_failure_count += 1
                if reason_for_schedule_failure in schedule_failure_reasons:
                    schedule_failure_reasons[reason_for_schedule_failure] += 1
                else:
                    schedule_failure_reasons[reason_for_schedule_failure] = 1

        if appointment_scheduled == "TRUE":
            appointment_scheduled_count += 1

    # Calculate percentages
    appointment_scheduled_percentage = (appointment_scheduled_count / make_appointment_count) * 100
    positive_percentage = (positive_count / make_appointment_count) * 100
    neutral_percentage = (neutral_count / make_appointment_count) * 100
    negative_percentage = (negative_count / make_appointment_count) * 100

#     # Print the results
#     print("Number of entries:", num_entries)
#     print("Number of 'positive' sentiments:", total_pos)
#     print("Number of 'negative' sentiments:", total_neg)
#     print("Number of 'neutral' sentiments:", total_neu)

#     print("\nNumber of times 'make_appointment' was the reason for calling:", make_appointment_count)
#     print("Number of times 'appointment_scheduled' was True:", appointment_scheduled_count)
#     print("Percentage of appointments scheduled: {:.2f}%".format(appointment_scheduled_percentage))

#     print("\nNumber of 'make_appointment' entries:", make_appointment_count)
#     print("Number of 'positive' sentiments specific to 'make_appointment':", positive_count)
#     print("Number of 'neutral' sentiments specific to 'make_appointment':", neutral_count)
#     print("Number of 'negative' sentiments specific to 'make_appointment':", negative_count)
#     print("Percentage of 'positive' sentiments specific to 'make_appointment': {:.2f}%".format(positive_percentage))
#     print("Percentage of 'neutral' sentiments specific to 'make_appointment': {:.2f}%".format(neutral_percentage))
#     print("Percentage of 'negative' sentiments specific to 'make_appointment': {:.2f}%".format(negative_percentage))

#     print("\nNumber of 'make_appointment' entries that resulted in 'appointment_scheduled: FALSE':", make_appointment_failure_count)
    for reason, count in schedule_failure_reasons.items():
        percentage = (count / make_appointment_failure_count) * 100
#         print("Reason for schedule failure:", reason)
#         print("Percentage: {:.2f}%".format(percentage))
        
    results = {
        "num_entries": num_entries,
        "total_pos": total_pos,
        "total_neg": total_neg,
        "total_neu": total_neu,
        "make_appointment_count": make_appointment_count,
        "appointment_scheduled_count": appointment_scheduled_count,
        "appointment_scheduled_percentage": appointment_scheduled_percentage,
        "positive_count": positive_count,
        "neutral_count": neutral_count,
        "negative_count": negative_count,
        "positive_percentage": positive_percentage,
        "neutral_percentage": neutral_percentage,
        "negative_percentage": negative_percentage,
        "make_appointment_failure_count": make_appointment_failure_count,
        "schedule_failure_reasons": schedule_failure_reasons
    }
    
    table1 = {
        "num_entries": results["num_entries"],
        "total_pos": results["total_pos"],
        "total_neg": results["total_neg"],
        "total_neu": results["total_neu"]
    }

    # Create a dictionary for the second table
    table2 = {
        "make_appointment_count": results["make_appointment_count"],
        "appointment_scheduled_count": results["appointment_scheduled_count"],
        "appointment_scheduled_percentage": results["appointment_scheduled_percentage"]
    }

    # Return the results
    return results

# ---------------------------------------------------------------------------------------
# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo-16k", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()
# ---------------------------------------------------------------------------------------

In [3]:
# Define a function to handle the chat interaction with the AI model
def chat(message):
    csv_file_path = 'data.csv'            
    directory = 'transcripts'
    all_json_data = []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are an AI assistant for a medical call center. The center's function is to take, record call information, and assist callers.
                
        ##Rules
        Don't exlcude any information, even if N/A.
        
        ##Analyze the transcribed audio and gather the following information:
        - name_of_patient
        - name_of_caller (patient or person on behalf of the patient.)
        - identity_of_caller [patient, patient_care_giver, insurance_representative, other_medical_office, other]
        - reason_for_calling [make_appointment, insurance_verification, change_appointment, cancel_appointment, request_test_results, request_perscription_refill, request_referral, inquiry_about_services, request_medical_records, provide_feedback, other]
        - appointment_scheduled [true, false, N/A] (True if appointment is successfully scheduled for a date, false if appointment cancelled or failed to schdule, N/A if not appilicable)
        - reason_for_schdule_failure [N/A, no_availability, no_referral, not_a_service, insurance_converage, incomplete_inforamtion, other]
        - problems_raised_by_caller (if no problem: N/A)
        - sentiment_of_call [positive, negative, neutral]
        """
        prompt2 = """
        You are a helpful JSON creator.

        Take the input and output it into JSON using the following keys and types:
        - name_of_patient [string]
        - name_of_caller [string]
        - identity_of_caller [string]
        - reason_for_calling [string]
        - appointment_scheduled [bool or null]
        - reason_for_schedule_failure [string or null]
        - problems_raised [string or null]
        - sentiment_of_call [string]
        """

        # Loop through each file in the directory
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                file_path = os.path.join(directory, filename)
                with open(file_path, 'r') as file:
                    file_content = file.read()
            
                    ai_reply = get_ai_reply(file_content, model="gpt-3.5-turbo-16k", system_message=prompt.strip())

                    # Append the user's message and the AI's reply to the history_state list
                    #history_state.append({"role": "user", "content": inputs[i]})
                    #history_state.append({"role": "assistant", "content": ai_reply})

                    ai_reply = get_ai_reply(ai_reply, model="gpt-3.5-turbo-16k", system_message=prompt2.strip())
                    #history_state.append({"role": "assistant", "content": ai_reply})

                    ##Add entry to CSV
                    if(is_valid_json(ai_reply)):
                        json_data = json.loads(ai_reply)
                        all_json_data.append(json_data)
        #data = [ { "name_of_patient": "Mark Johnson", "name_of_caller": "Mark Johnson", "identity_of_caller": "patient", "reason_for_calling": "provide_feedback", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "long wait time, doctor seemed rushed and didn't listen to concerns", "sentiment_of_call": "negative" }, { "name_of_patient": "Sarah Johnson", "name_of_caller": "Sarah Johnson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "John Smith", "name_of_caller": "John Smith", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_referral", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "change_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "frustration with lack of response and inconvenience caused", "sentiment_of_call": "negative" }, { "name_of_patient": "John Smith", "name_of_caller": "Lisa", "identity_of_caller": "insurance_representative", "reason_for_calling": "insurance_verification, discuss_billing_process", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "Emma Thompson", "name_of_caller": "Mrs. Thompson", "identity_of_caller": "patient_care_giver", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "", "name_of_caller": "", "identity_of_caller": "", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "Mark Johnson", "name_of_caller": "Mark Johnson", "identity_of_caller": "patient", "reason_for_calling": "provide_feedback", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "long wait time, doctor seemed rushed and didn't listen to concerns", "sentiment_of_call": "negative" }, { "name_of_patient": "Sarah Johnson", "name_of_caller": "Sarah Johnson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "John Smith", "name_of_caller": "John Smith", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_referral", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "change_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "frustration with lack of response and inconvenience caused", "sentiment_of_call": "negative" }, { "name_of_patient": "John Smith", "name_of_caller": "Lisa", "identity_of_caller": "insurance_representative", "reason_for_calling": "insurance_verification, discuss_billing_process", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "Emma Thompson", "name_of_caller": "Mrs. Thompson", "identity_of_caller": "patient_care_giver", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "inquiry_about_services", "appointment_scheduled": "", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "TRUE", "reason_for_schedule_failure": "", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "not_a_service", "problems_raised": "", "sentiment_of_call": "positive" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "negative" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" }, { "name_of_patient": "James Thompson", "name_of_caller": "James Thompson", "identity_of_caller": "patient", "reason_for_calling": "make_appointment", "appointment_scheduled": "FALSE", "reason_for_schedule_failure": "no_availability", "problems_raised": "", "sentiment_of_call": "neutral" } ]
        raw = pd.DataFrame(data)
        raw.to_csv()
        results = EXTRACT_INFO(data)
        df = pd.DataFrame([results])
        
        num_entries = f"<h1>num_entries:</h1>\n<h2>{df['num_entries'].values[0]}</h2>"
        total_pos = f"<h1>total_positives:</h1>\n<h2>{df['total_pos'].values[0]}</h2>"
        total_neg = f"<h1>total_negatives:</h1>\n<h2>{df['total_neg'].values[0]}</h2>"
        total_neu = f"<h1>total_nerutrals</h1>\n<h2>{df['total_neu'].values[0]}</h2>"

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return num_entries, total_pos, total_neg, total_neu, raw, csv_file_path

# Define a function to launch the chatbot interface using Gradio
def get_chatbot_app():
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        # Create a textbox for the user's message
        message = gr.Textbox(label="Message")
        
        num_entries = gr.HTML()
        total_pos = gr.HTML()
        total_neg = gr.HTML()
        total_neu = gr.HTML()
        csv = gr.Dataframe()
        csv_file = gr.File(label="Download (CSV)", interactive=False)
        
        btn = gr.Button(value="Send")
        # Connect the send button to the chat function
        btn.click(chat, inputs=[message], outputs=[num_entries, total_pos, total_neg, total_neu, csv, csv_file])
        # Return the app
        return app
# ---------------------------------------------------------------------------------------        
# Call the launch_chatbot function to start the chatbot interface using Gradio
app = get_chatbot_app()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4b847df701d06d7401.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
